#### Install and import mediapipe and opencv

In [1]:
#pip install mediapipe opencv-python 
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils #gives us drawing utilities
mp_pose = mp.solutions.pose #the pose estimation model

We're gonna be using the below block of code again, it's a basic framework to take webcam input and show it in a window

In [2]:
#Getting a real time video feed from our webcam
cap = cv2.VideoCapture(0)   #capturing video from our webcam: the parameter is usually 0, otherwise chnge it around and see
while cap.isOpened():
    frame = cap.read()[1]   #reading what you've captured from webcam, it returns return value and the frame content
    cv2.imshow('Mediapipe Feed',frame)  #showing it in a window

    if cv2.waitKey(10) & 0xFF == ord('q'):  #taking the 0xFF bits of keyboard input(ord gives unicode of character): if q close after 10 millisecs
        break

cap.release()   #release the video feed capturing
cv2.destroyAllWindows() #close windows opened

#it didn't close even when i pressed x button, only when i entered q(not Q)

#### Make Detections

In [10]:
cap = cv2.VideoCapture(0)  

# setting up a mediapipe instance mp_pose was defined earlier remember
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:    #this confidence levels can be increased as per convenvience
    while cap.isOpened():
        frame = cap.read()[1]   #datatype is numpy array

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)   #mediapipe requires RBG, but when captured its BGR
        image.flags.writeable = False   #to make the RGB colour arrays editable i think

        results = pose.process(image)   #access our pose model to process our image and store the detections in results

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #bring it back to BGR so we can rerender it using cv2

        #now we draw on our image with the landmarks: all the body tracking points, and then add the connections between thme as well
        # and change the colours(BGR) for landmark first and then connection as well
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,255,0),thickness=2,circle_radius=2)
            )

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# mp_drawing.draw_landmarks?? 

#gives deets regarding this object

#### Determining Joints

We build upon the same code and just get the landmarks and their coordinates in a try except statement

In [12]:
cap = cv2.VideoCapture(0)  

# i increased the confidence
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        frame = cap.read()[1]

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # get the landmarks(this is what we passed to draw below), 
        # some joints might be out of camera etc, so otherwie pass
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,255,0),thickness=2,circle_radius=2)
            )

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Now we see that there are 33 landmarks and they are as follows. This allows us to choose the ones we require.

In [13]:
len(landmarks)

33

In [14]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk, lndmrk.value)

PoseLandmark.NOSE 0
PoseLandmark.LEFT_EYE_INNER 1
PoseLandmark.LEFT_EYE 2
PoseLandmark.LEFT_EYE_OUTER 3
PoseLandmark.RIGHT_EYE_INNER 4
PoseLandmark.RIGHT_EYE 5
PoseLandmark.RIGHT_EYE_OUTER 6
PoseLandmark.LEFT_EAR 7
PoseLandmark.RIGHT_EAR 8
PoseLandmark.MOUTH_LEFT 9
PoseLandmark.MOUTH_RIGHT 10
PoseLandmark.LEFT_SHOULDER 11
PoseLandmark.RIGHT_SHOULDER 12
PoseLandmark.LEFT_ELBOW 13
PoseLandmark.RIGHT_ELBOW 14
PoseLandmark.LEFT_WRIST 15
PoseLandmark.RIGHT_WRIST 16
PoseLandmark.LEFT_PINKY 17
PoseLandmark.RIGHT_PINKY 18
PoseLandmark.LEFT_INDEX 19
PoseLandmark.RIGHT_INDEX 20
PoseLandmark.LEFT_THUMB 21
PoseLandmark.RIGHT_THUMB 22
PoseLandmark.LEFT_HIP 23
PoseLandmark.RIGHT_HIP 24
PoseLandmark.LEFT_KNEE 25
PoseLandmark.RIGHT_KNEE 26
PoseLandmark.LEFT_ANKLE 27
PoseLandmark.RIGHT_ANKLE 28
PoseLandmark.LEFT_HEEL 29
PoseLandmark.RIGHT_HEEL 30
PoseLandmark.LEFT_FOOT_INDEX 31
PoseLandmark.RIGHT_FOOT_INDEX 32


So to get the coordinates of a particular landmark we do the following

In [15]:
print(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
#or
print(landmarks[11])

x: 0.7357970476150513
y: 0.8371623158454895
z: -0.07982823997735977
visibility: 0.9987908005714417

x: 0.7357970476150513
y: 0.8371623158454895
z: -0.07982823997735977
visibility: 0.9987908005714417



#### Calculating angles

In [5]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [6]:
shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]


In [7]:
calculate_angle(shoulder_left,elbow_left,wrist_left)

30.454702929966423

In [43]:
cap = cv2.VideoCapture(0)  

# i decreased the confidence because landmarks weren't generating
with mp_pose.Pose(min_detection_confidence=0.7,min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        frame = cap.read()[1]

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        try:
            landmarks = results.pose_landmarks.landmark

            # get coordinates for angle calculation
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            thumb_right = [landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y]
            # wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            # elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            angle1 = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angle2 = calculate_angle(thumb_right, wrist_right, elbow_right)

            # visualize this angle by showing angle value on screen as a string
            # the coordinates we have are relative coordinates so we multiply them with
            # the size of our window            
            cv2.putText(image, str(angle1),
                    tuple(np.multiply(elbow_right, [640,480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA )
            cv2.putText(image, str(angle2),
                    tuple(np.multiply(wrist_right, [640,480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA )

        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,255,0),thickness=2,circle_radius=2)
            )

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Be mindful of the midpoint and text on the midpoint when you change the landmark to find angles for different joints

#### Curl counter

In [8]:
cap = cv2.VideoCapture(0)  

#add a count variable and a stage variable
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.7,min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        frame = cap.read()[1]

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        try:
            landmarks = results.pose_landmarks.landmark

            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            thumb_right = [landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y]

            angle1 = calculate_angle(shoulder_right, elbow_right, wrist_right)
            # angle2 = calculate_angle(thumb_right, wrist_right, elbow_right)

                       
            cv2.putText(image, str(angle1),
                    tuple(np.multiply(elbow_right, [640,480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA )
            # cv2.putText(image, str(angle2),
            #         tuple(np.multiply(wrist_right, [640,480]).astype(int)),
            #         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA )

            # curl counter based on angles
            if angle1 > 160:
                stage = "down"
            elif angle1 < 40 and stage == 'down':
                stage = "up"
                counter += 1
                print(counter)

        except:
            pass
        
        # Render curl counter: status box is placed in top left corner
        # parameters are image, start and end point of rectangle, color, line width/thickness (-1 fills the box)
        cv2.rectangle(image, (0,0), (225, 73), (245,117,16), -1)

        # Data about the curls
        # image, text we want to display, start coordinate of text, font, size etc. and same for counter
        cv2.putText(image, 'REPS', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        #stage of rep also is displayed as below
        cv2.putText(image, 'STAGE', (65,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (70,60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
            mp_drawing.DrawingSpec(color=(0,255,0),thickness=2,circle_radius=2)
            )

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1
2
3
4
5


In [36]:
#cv2.rectangle??

Docstring:
rectangle(img, pt1, pt2, color[, thickness[, lineType[, shift]]]) -> img
.   @brief Draws a simple, thick, or filled up-right rectangle.
.   
.   The function cv::rectangle draws a rectangle outline or a filled rectangle whose two opposite corners
.   are pt1 and pt2.
.   
.   @param img Image.
.   @param pt1 Vertex of the rectangle.
.   @param pt2 Vertex of the rectangle opposite to pt1 .
.   @param color Rectangle color or brightness (grayscale image).
.   @param thickness Thickness of lines that make up the rectangle. Negative values, like #FILLED,
.   mean that the function has to draw a filled rectangle.
.   @param lineType Type of the line. See #LineTypes
.   @param shift Number of fractional bits in the point coordinates.



rectangle(img, rec, color[, thickness[, lineType[, shift]]]) -> img
.   @overload
.   
.   use `rec` parameter as alternative specification of the drawn rectangle: `r.tl() and
.   r.br()-Point(1,1)` are opposite corners
Type:      builtin_function